In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

# CREATING PARAMETER

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag =dbutils.widgets.get('incremental_flag')

# CREATING DIMENSION

In [0]:
df_src=spark.sql('''select DISTINCT Model_ID, model_catagory
from parquet.`abfss://silver@carsaleslake.dfs.core.windows.net/carsales`''')

In [0]:
df_src.display()

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    df_sink=spark.sql('''
    SELECT dim_model_key,Model_ID, model_catagory
    FROM cars_catalog.gold.dim_model
    ''') 
else:
    df_sink=spark.sql('''
    SELECT 1 as dim_model_key,Model_ID, model_catagory
    FROM parquet.`abfss://silver@carsaleslake.dfs.core.windows.net/carsales`
    WHERE 1=0''')

### Filtering new and old records

In [0]:
df_filter=df_src.join(df_sink, df_src['Model_ID']==df_sink['Model_ID'],'left').select(df_src['Model_ID'], df_src['model_catagory'], df_sink['dim_model_key'])

**df_filter_old** 

In [0]:
df_filter_old=df_filter.filter(col('dim_model_key').isNotNull())

**df_filter_new**

In [0]:
df_filter_new=df_filter.filter(col('dim_model_key').isNull()).select(df_src['Model_ID'], df_src['model_catagory'])

In [0]:
df_filter_new.display()

#Create Surrogate Key

**Fetch max Surrogate Key from existing table**

In [0]:
if (incremental_flag == '0'):
    max_value =0
else:
    max_value = spark.sql("select max(dim_model_key) from cars_catalog.gold.dim_model")
    max_value = max_value-df.collect()[0][0]

**Create Surrogate key column and ADD the max surrogate key**

In [0]:
df_filter_new=df_filter_new.withColumn("dim_model_key", max_value + monotonically_increasing_id())

In [0]:
df_filter_new.display()

## Creating final DF (Old + New)

In [0]:
df_final= df_filter_new.union(df_filter_old)

In [0]:
df_final.display()

# SCD TYPE -1 (UPSERT)

In [0]:
from delta.tables import DeltaTable

In [0]:
#Incremental Run
if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    delta_tbl= DeltaTable.forPath(spark,"abfss://gold@carsaleslake.dfs.core.windows.net/dim_model")

    delta_tbl.alias("trg").merge(df_final.alias("src"), "trg.dim_model_key = src.dim_model_key")\
                            .whenMatchedUpdateAll()\
                            .whenNotMatchedInsertAll()\
                               .execute()
#Initial run
else:
    df_final.write.format("delta").mode("overwrite").option("path","abfss://gold@carsaleslake.dfs.core.windows.net/dim_model").saveAsTable("cars_catalog.gold.dim_model")

In [0]:
%sql
SELECT * FROM cars_catalog.gold.dim_model